In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import requests
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
columns = ['url', 'seller_url', 'title', 'rating', 'final_price', 'old_price', 'info', 'description']
wb = pd.DataFrame(columns=columns)

In [ ]:
driver = webdriver.Chrome()
hrefs = []
n = 'input your number of pages'

try:
    for page in range(1,3):
        driver.get(url=f'[-PUT-YOUR-WB-URL-HERE-]?sort=popular&page={page}')
        time.sleep(10)
        print('main base loaded')
    #   Постепенная прогрузка страницы
        skrol = 0
        for i in range(15):
            skrol += 1000
            driver.execute_script(f"window.scrollTo(0, {skrol});")
            time.sleep(1)
        time.sleep(10)
        print(f'page {page} loaded')
    #   Взятие href с каждой страницы  
        for i in range(1, 101):
                    try:                                       
                        item = driver.find_element(By.XPATH, f'/html/body/div[1]/main/div[2]/div/div[2]/div/div[5]/div[1]/div[1]/div/article[{i}]/div/a').get_attribute('href')
                        hrefs.append(item)
                        time.sleep(1)
                    except Exception as inner_ex:
                        print(f'Error in inner loop: {inner_ex}')
                        continue

except Exception as ex:
    print(f'Error in outer loop: {ex}')

driver.close()
driver.quit()

In [ ]:
driver = webdriver.Chrome()
data_spec_all = {}

try:
    for k in range(len(hrefs)+1):
        
#         Инициализация селениума
        driver.get(url=hrefs[k])
        time.sleep(10)
#         Название товара
        title = driver.find_element(By.CLASS_NAME, 'product-page__title').text
#         Цены товара
        final_price = driver.find_element(By.CLASS_NAME,"price-block__final-price")
        old_price = driver.find_element(By.CLASS_NAME,"price-block__old-price")
#         Рейтинг товара
        try:
            rating_element = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[8]/div[3]/a/span[1]')
            rating = rating_element.text
        except Exception as rating_exception:
            rating = None
#         Url продавца
        seller = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[8]/div[1]/a').get_attribute('href') 
#         Кнопка описания
        button = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[3]/div/div[3]/div[7]/button')
        driver.execute_script("arguments[0].click();", button)
        time.sleep(1)
#         Описание
        try:
            description = driver.find_element(By.CLASS_NAME, 'option__text').text
        except Exception as description_exception:
            description = None
#         Таблицы с характеристиками
        data_spec = []
        table = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div[2]/table')
        
        # Обработка данных таблицы, если она найдена
        if table:
            table_body = table.find_element(By.TAG_NAME, 'tbody')
            caption = f'{k}'
            for tr in table_body.find_elements(By.TAG_NAME, 'tr'):
                char = tr.find_element(By.TAG_NAME, 'th').text
                value = tr.find_element(By.TAG_NAME, 'td').text
                data_spec.append([char.strip(), value.strip()])
            data_spec_all[caption] = data_spec
            
        
# Добавляем данные с url товара
            new_data = {'url': hrefs[k],'seller_url':seller,'title':title,'rating':rating, 'final_price': final_price.text,
                        'old_price': old_price.text, 'info': data_spec_all[caption],'description': description}
            wb = wb.append(new_data, ignore_index=True)
        else:
            print(f'Table not found for {hrefs[k]}')
        time.sleep(5)
except Exception as ex:
    print(ex)
finally:
    driver.close()
    driver.quit()   

In [ ]:
wb.head()

In [ ]:
wb.to_csv('wb.csv', encoding='utf-8')